## Post Import in Group

This demonstrates the process of deploying a new Power BI Model as a PBIX file.  This example loads the file in the body of the API call as encoded binary data.

The documentation for the relevant APIs can be found here:
- https://learn.microsoft.com/en-us/rest/api/power-bi/imports/create-temporary-upload-location-in-group
- https://learn.microsoft.com/en-us/rest/api/power-bi/imports/post-import-in-group

In [ ]:
pip install requests msal jsons azure-storage-blob requests_toolbelt

In [ ]:
#For orgaization purposes I put notbooks in subfolders not the root of the proejct.aad_token
#This code adds the root directory of the project to the sys path so we can load class modules from the services folder
#I think this only needs to be run once, but including it for completeness.
import os, sys
projectRoot = os.path.abspath('.')
directory = os.path.dirname(projectRoot)
if not directory in sys.path: sys.path.append(directory)

In [ ]:
#This leverages the code encapsulated in services/aadservice.py that encapsulates the service principle login
from services.aadservice import AadService
aad_token = AadService.get_access_token()

In [ ]:
workspace_id="16453fea-cb0f-4ef4-b1b2-bede9d3b92be"
display_name="AdventureWorks.pbix"

In [ ]:
import requests
from requests_toolbelt.multipart.encoder import MultipartEncoder

url = f'https://api.powerbi.com/v1.0/myorg/groups/{workspace_id}/imports?datasetDisplayName={display_name}'
headers = {
    'Content-Type': 'multipart/form-data',
    'authorization': 'Bearer ' + aad_token
}

# you need this dictionary to convert a binary file into form-data format
# None here means we skip the filename and file content is important 
files = {'value': (None, open(file_location, 'rb'), 'multipart/form-data')}

mp_encoder = MultipartEncoder(fields=files)
print(mp_encoder)
r = requests.post(
    url=url,
    data=mp_encoder,  # The MultipartEncoder is posted as data, don't use files=...!
    # The MultipartEncoder provides the content-type header with the boundary:
    headers=headers
)
